In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import datetime
import gspread
import time
import pandas as pd


In [44]:
driver = webdriver.Chrome()
url = 'https://www.nseindia.com/market-data/new-stock-exchange-listings-recent'
driver.get(url)
time.sleep(10)
r = driver.page_source
driver.close()

In [45]:
soup= bs(r,'html.parser') 
if any(table.get('id') == "livenltRecentTable" for table in soup.find_all('table')):
    Final_Table = []
    DataInTable = []
    table = soup.find('div',{'class':'customTable-width tableWidth-1150 firstRow deque-table-sortable-group'})
    #print(table)
    for rows in table:
        row = rows.find_all('td')
    for data in row[1:]:
        DataInTable.append(data.text)
    for i in range(0,len(DataInTable),6):
        New=DataInTable[i:i+6]
        Final_Table.append(New)
    print(Final_Table)
else:
    print("Something")
    

[['Equity', 'HDFCPSUBK', 'HDFC Mutual Fund -HDFC NIFTY PSU BANK ETF', 'EQ', 'INF179KC1HW4', '07-Feb-2024'], ['Equity', 'SMFG', 'SMFG India Credit Company Limited', 'N2', 'INE535H07CE7', '07-Feb-2024'], ['Equity', 'HEALTHADD', 'DSP Mutual Fund - DSP Nifty Healthcare ETF', 'EQ', 'INF740KA1UF4', '06-Feb-2024'], ['SME', 'BAWEJA', 'Baweja Studios Limited', 'ST', 'INE0JFJ01011', '06-Feb-2024'], ['Equity', 'HEFC', 'Hero FinCorp Limited', 'N1', 'INE957N07773', '06-Feb-2024'], ['Equity', 'BLSE', 'BLS E-Services Limited', 'EQ', 'INE0NLT01010', '05-Feb-2024'], ['SME', 'MEGATHERM', 'Megatherm Induction Limited', 'ST', 'INE531R01010', '02-Feb-2024'], ['Equity', 'CHOLAFIN', 'Cholamandalam Investment and Finance Company Limited', 'NJ', 'INE121A07RQ3', '02-Feb-2024'], ['Equity', 'NIF5GETF', 'UTI Mutual Fund - UTI Nifty 5 yr Benchmark G-Sec ETF', 'EQ', 'INF789F1AZE6', '02-Feb-2024'], ['Equity', 'NIF10GETF', 'UTI Mutual Fund - UTI Nifty 10 yr Benchmark G-Sec ETF', 'EQ', 'INF789F1AZF3', '02-Feb-2024'], [

In [46]:
print(DataInTable)

['Equity', 'HDFCPSUBK', 'HDFC Mutual Fund -HDFC NIFTY PSU BANK ETF', 'EQ', 'INF179KC1HW4', '07-Feb-2024', 'Equity', 'SMFG', 'SMFG India Credit Company Limited', 'N2', 'INE535H07CE7', '07-Feb-2024', 'Equity', 'HEALTHADD', 'DSP Mutual Fund - DSP Nifty Healthcare ETF', 'EQ', 'INF740KA1UF4', '06-Feb-2024', 'SME', 'BAWEJA', 'Baweja Studios Limited', 'ST', 'INE0JFJ01011', '06-Feb-2024', 'Equity', 'HEFC', 'Hero FinCorp Limited', 'N1', 'INE957N07773', '06-Feb-2024', 'Equity', 'BLSE', 'BLS E-Services Limited', 'EQ', 'INE0NLT01010', '05-Feb-2024', 'SME', 'MEGATHERM', 'Megatherm Induction Limited', 'ST', 'INE531R01010', '02-Feb-2024', 'Equity', 'CHOLAFIN', 'Cholamandalam Investment and Finance Company Limited', 'NJ', 'INE121A07RQ3', '02-Feb-2024', 'Equity', 'NIF5GETF', 'UTI Mutual Fund - UTI Nifty 5 yr Benchmark G-Sec ETF', 'EQ', 'INF789F1AZE6', '02-Feb-2024', 'Equity', 'NIF10GETF', 'UTI Mutual Fund - UTI Nifty 10 yr Benchmark G-Sec ETF', 'EQ', 'INF789F1AZF3', '02-Feb-2024', 'Equity', 'MUTHOOTFIN

In [51]:
df = pd.DataFrame(Final_Table,columns=['Market Type','Symbol','Company Name','Series','ISIN','Date'])
filtered_df = df[df['Series'].str.contains('EQ|ST|BE|MF|GB')]
filtered_df.head()


,Market Type,Symbol,Company Name,Series,ISIN,Date
0,Equity,HDFCPSUBK,HDFC Mutual Fund -HDFC NIFTY PSU BANK ETF,EQ,INF179KC1HW4,07-Feb-2024
2,Equity,HEALTHADD,DSP Mutual Fund - DSP Nifty Healthcare ETF,EQ,INF740KA1UF4,06-Feb-2024
3,SME,BAWEJA,Baweja Studios Limited,ST,INE0JFJ01011,06-Feb-2024
5,Equity,BLSE,BLS E-Services Limited,EQ,INE0NLT01010,05-Feb-2024
6,SME,MEGATHERM,Megatherm Induction Limited,ST,INE531R01010,02-Feb-2024


In [48]:
gc = gspread.service_account(filename='creds.json')
spreadsheet_name = 'Notifications and Listings'
sheet_name = 'NSE_Listing'
sh = gc.open(spreadsheet_name).worksheet(sheet_name)
existing_data_range = sh.range('A2:L' + str(sh.row_count))
for cell in existing_data_range:
    cell.value = ''

sh.update_cells(existing_data_range)

{'spreadsheetId': '1Otk1m6VxGHQ5nKa5z9bNg5ebTEu7w5k6j4dsSLx5I3s',
 'updatedRange': 'NSE_Listing!A2:L1000',
 'updatedRows': 999,
 'updatedColumns': 12,
 'updatedCells': 11988}

In [49]:
values_lists = []
df_dict = filtered_df.to_dict(orient='records')
for data in df_dict:
    values_lists.append([str(data['Date']),str(data['Market Type']), str(data['Symbol']), str(data['Company Name']),str(data['Series']),str(data['ISIN'])])
sh.append_row([f'Batch ran at: {datetime.datetime.now()}'])
sh.append_rows(values_lists) 

{'spreadsheetId': '1Otk1m6VxGHQ5nKa5z9bNg5ebTEu7w5k6j4dsSLx5I3s',
 'tableRange': 'NSE_Listing!A1:F2',
 'updates': {'spreadsheetId': '1Otk1m6VxGHQ5nKa5z9bNg5ebTEu7w5k6j4dsSLx5I3s',
  'updatedRange': 'NSE_Listing!A3'}}